Data Preprocessing 

In [1]:
import pydicom
import os
import numpy as np
import cv2

def load_dicom_images(folder_path):
    images = []
    for subdir, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".dcm"):
                dicom_path = os.path.join(subdir, file)
                dicom = pydicom.dcmread(dicom_path)
                image = dicom.pixel_array
                image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX)  # Normalize
                image = cv2.resize(image, (224, 224))  # Resize for FRCNN
                images.append(image) 
    return np.array(images)

# Example usage
folder_path = "E:\FinalYearProject\Code\RaFoDATA\DatasetRF\dcm images\manifest-1608669183333\Lung-PET-CT-Dx"
images = load_dicom_images(folder_path)

<>:20: SyntaxWarning: invalid escape sequence '\F'
<>:20: SyntaxWarning: invalid escape sequence '\F'
C:\Users\shivam\AppData\Local\Temp\ipykernel_4460\1563612700.py:20: SyntaxWarning: invalid escape sequence '\F'
  folder_path = "E:\FinalYearProject\Code\RaFoDATA\DatasetRF\dcm images\manifest-1608669183333\Lung-PET-CT-Dx"
C:\Users\shivam\AppData\Local\Temp\ipykernel_4460\1563612700.py:20: SyntaxWarning: invalid escape sequence '\F'
  folder_path = "E:\FinalYearProject\Code\RaFoDATA\DatasetRF\dcm images\manifest-1608669183333\Lung-PET-CT-Dx"


KeyboardInterrupt: 

In [1]:
import xml.etree.ElementTree as ET
import os

def parse_xml_annotation(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    boxes = []
    labels = []

    for obj in root.findall('object'):
        label = obj.find('name').text
        bndbox = obj.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)
        boxes.append([xmin, ymin, xmax, ymax])
        labels.append(label)

    return boxes, labels

# Example usage
xml_file = "E:\FinalYearProject\Code\RaFoDATA\DatasetRF\Annotation"
boxes, labels = parse_xml_annotation(xml_file)
print("Bounding Boxes:", boxes)
print("Labels:", labels)

<>:24: SyntaxWarning: invalid escape sequence '\F'
<>:24: SyntaxWarning: invalid escape sequence '\F'
C:\Users\shivam\AppData\Local\Temp\ipykernel_10104\3340822199.py:24: SyntaxWarning: invalid escape sequence '\F'
  xml_file = "E:\FinalYearProject\Code\RaFoDATA\DatasetRF\Annotation"
C:\Users\shivam\AppData\Local\Temp\ipykernel_10104\3340822199.py:24: SyntaxWarning: invalid escape sequence '\F'
  xml_file = "E:\FinalYearProject\Code\RaFoDATA\DatasetRF\Annotation"


PermissionError: [Errno 13] Permission denied: 'E:\\FinalYearProject\\Code\\RaFoDATA\\DatasetRF\\Annotation'

In [ ]:
import numpy as np

def load_data(image_folder, annotation_folder):
    images = []
    all_boxes = []
    all_labels = []

    for xml_file in os.listdir(annotation_folder):
        if xml_file.endswith(".xml"):
            # Parse XML annotation
            xml_path = os.path.join(annotation_folder, xml_file)
            boxes, labels = parse_xml_annotation(xml_path)

            # Load corresponding DICOM image
            image_name = os.path.splitext(xml_file)[0] + ".dcm"
            image_path = os.path.join(image_folder, image_name)
            dicom = pydicom.dcmread(image_path)
            image = dicom.pixel_array
            image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX)  # Normalize
            image = cv2.resize(image, (224, 224))  # Resize for FRCNN

            # Append to lists
            images.append(image)
            all_boxes.append(boxes)
            all_labels.append(labels)

    # Convert to numpy arrays
    images = np.array(images)
    all_boxes = np.array(all_boxes)
    all_labels = np.array(all_labels)

    return images, all_boxes, all_labels

# Example usage
image_folder = "E:\FinalYearProject\Code\RaFoDATA\DatasetRF\dcm images\manifest-1608669183333\Lung-PET-CT-Dx"
annotation_folder = "E:\FinalYearProject\Code\RaFoDATA\DatasetRF\Annotation"
images, boxes, labels = load_data(image_folder, annotation_folder)

In [ ]:
import pandas as pd

# Example: Load annotations from a CSV file
annotations = pd.read_excel("E:\FinalYearProject\Code\RaFoDATA\DatasetRF\Annotation")

 Process DICOM and XML Files

In [ ]:
import os
import pydicom
import cv2
import xml.etree.ElementTree as ET

# Define paths
dicom_root = r"E:\FinalYearProject\Code\RaFoDATA\DatasetRF\dcm images\manifest-1608669183333\Lung-PET-CT-Dx"
annotation_root = r"E:\FinalYearProject\Code\RaFoDATA\DatasetRF\Annotation"
output_root = r"E:\FinalYearProject\Code\RaFoDATA\DatasetRF\Processed"

# Create output folder if it doesn't exist
os.makedirs(output_root, exist_ok=True)

def load_dicom_image(dicom_path):
    """Load a DICOM image and return it as a NumPy array."""
    dicom = pydicom.dcmread(dicom_path)
    image = dicom.pixel_array
    return image

def parse_xml_annotation(xml_path):
    """Parse XML annotations and extract bounding boxes and labels."""
    tree = ET.parse(xml_path)
    root = tree.getroot()
    boxes = []
    labels = []
    for obj in root.findall('object'):
        label = obj.find('name').text
        bbox = obj.find('bndbox')
        xmin = int(bbox.find('xmin').text)
        ymin = int(bbox.find('ymin').text)
        xmax = int(bbox.find('xmax').text)
        ymax = int(bbox.find('ymax').text)
        boxes.append([xmin, ymin, xmax, ymax])
        labels.append(label)
    return boxes, labels

def convert_dicom_to_png(dicom_path, output_path):
    """Convert a DICOM image to PNG format and save it."""
    image = load_dicom_image(dicom_path)
    cv2.imwrite(output_path, image)

def process_dataset(dicom_root, annotation_root, output_root):
    """Process all DICOM images and XML annotations in the dataset."""
    for patient_folder in os.listdir(dicom_root):
        patient_path = os.path.join(dicom_root, patient_folder)
        if os.path.isdir(patient_path):
            for study_folder in os.listdir(patient_path):
                study_path = os.path.join(patient_path, study_folder)
                if os.path.isdir(study_path):
                    for series_folder in os.listdir(study_path):
                        series_path = os.path.join(study_path, series_folder)
                        if os.path.isdir(series_path):
                            for filename in os.listdir(series_path):
                                if filename.endswith(".dcm"):
                                    dicom_path = os.path.join(series_path, filename)
                                    xml_filename = filename.replace(".dcm", ".xml")
                                    xml_path = os.path.join(annotation_root, xml_filename)
                                    
                                    # Check if XML annotation exists
                                    if not os.path.exists(xml_path):
                                        print(f"Warning: No annotation found for {filename}")
                                        continue
                                    
                                    # Create output folder for the patient
                                    output_patient_folder = os.path.join(output_root, patient_folder)
                                    os.makedirs(output_patient_folder, exist_ok=True)
                                    
                                    # Convert DICOM to PNG
                                    output_image_path = os.path.join(output_patient_folder, filename.replace(".dcm", ".png"))
                                    convert_dicom_to_png(dicom_path, output_image_path)
                                    
                                    # Parse XML annotations
                                    boxes, labels = parse_xml_annotation(xml_path)
                                    print(f"Processed {filename}: Boxes={boxes}, Labels={labels}")

# Run the processing script
process_dataset(dicom_root, annotation_root, output_root)
print("Dataset processing complete!")

Dataset processing complete!


In [2]:
import os
import pydicom
import cv2
import xml.etree.ElementTree as ET
from multiprocessing import Pool, cpu_count

# Define paths (NVMe SSD)
dicom_root = r"E:\LungCancerDataset\dcm images\manifest-1608669183333\Lung-PET-CT-Dx"
annotation_root = r"E:\LungCancerDataset\Annotation"
output_root = r"E:\LungCancerDataset\Processed"

# Create output folder if it doesn't exist
os.makedirs(output_root, exist_ok=True)

def load_dicom_image(dicom_path):
    """Load a DICOM image and return it as a NumPy array."""
    dicom = pydicom.dcmread(dicom_path)
    image = dicom.pixel_array
    return image

def parse_xml_annotation(xml_path):
    """Parse XML annotations and extract bounding boxes and labels."""
    tree = ET.parse(xml_path)
    root = tree.getroot()
    boxes = []
    labels = []
    for obj in root.findall('object'):
        label = obj.find('name').text
        bbox = obj.find('bndbox')
        xmin = int(bbox.find('xmin').text)
        ymin = int(bbox.find('ymin').text)
        xmax = int(bbox.find('xmax').text)
        ymax = int(bbox.find('ymax').text)
        boxes.append([xmin, ymin, xmax, ymax])
        labels.append(label)
    return boxes, labels

def convert_dicom_to_png(dicom_path, output_path):
    """Convert a DICOM image to PNG format and save it."""
    image = load_dicom_image(dicom_path)
    cv2.imwrite(output_path, image)

def find_xml_file(dicom_filename, annotation_root):
    """Recursively search for the corresponding XML file in subfolders."""
    for root, dirs, files in os.walk(annotation_root):
        for file in files:
            if file == dicom_filename.replace(".dcm", ".xml"):
                return os.path.join(root, file)
    return None

def process_file(args):
    dicom_path, annotation_root, output_root = args
    try:
        # Find the corresponding XML file
        xml_path = find_xml_file(os.path.basename(dicom_path), annotation_root)
        if not xml_path:
            print(f"Warning: No annotation found for {os.path.basename(dicom_path)}")
            return
        
        # Create output folder for the patient
        patient_folder = os.path.basename(os.path.dirname(os.path.dirname(dicom_path)))
        output_patient_folder = os.path.join(output_root, patient_folder)
        os.makedirs(output_patient_folder, exist_ok=True)
        
        # Convert DICOM to PNG
        output_image_path = os.path.join(output_patient_folder, os.path.basename(dicom_path).replace(".dcm", ".png"))
        convert_dicom_to_png(dicom_path, output_image_path)
        
        # Parse XML annotations
        boxes, labels = parse_xml_annotation(xml_path)
        print(f"Processed {os.path.basename(dicom_path)}: Boxes={boxes}, Labels={labels}")
    except Exception as e:
        print(f"Error processing {dicom_path}: {e}")

def process_dataset_parallel(dicom_root, annotation_root, output_root):
    """Process all DICOM images and XML annotations in parallel."""
    tasks = []
    for patient_folder in os.listdir(dicom_root):
        patient_path = os.path.join(dicom_root, patient_folder)
        if os.path.isdir(patient_path):
            for study_folder in os.listdir(patient_path):
                study_path = os.path.join(patient_path, study_folder)
                if os.path.isdir(study_path):
                    for series_folder in os.listdir(study_path):
                        series_path = os.path.join(study_path, series_folder)
                        if os.path.isdir(series_path):
                            for filename in os.listdir(series_path):
                                if filename.endswith(".dcm"):
                                    dicom_path = os.path.join(series_path, filename)
                                    tasks.append((dicom_path, annotation_root, output_root))
    
    # Use multiprocessing to process files in parallel
    with Pool(cpu_count()) as pool:
        pool.map(process_file, tasks)

# Run the parallel processing script
process_dataset_parallel(dicom_root, annotation_root, output_root)
print("Dataset processing complete!")

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'E:\\LungCancerDataset\\dcm images\\manifest-1608669183333\\Lung-PET-CT-Dx'

In [5]:
import os

dicom_root = r"E:\FinalYearProject\Code\RaFoDATA\DatasetRF\dcm images\manifest-1608669183333\Lung-PET-CT-Dx\Lung_Dx-A0001\04-04-2007-NA-Chest-07990\2.000000-5mm-40805"
if not os.path.exists(dicom_root):
    print(f"Error: The folder '{dicom_root}' does not exist.")
else:
    print(f"Folder '{dicom_root}' exists. Contents:")
    print(os.listdir(dicom_root))

Folder 'E:\FinalYearProject\Code\RaFoDATA\DatasetRF\dcm images\manifest-1608669183333\Lung-PET-CT-Dx\Lung_Dx-A0001\04-04-2007-NA-Chest-07990\2.000000-5mm-40805' exists. Contents:
['1-01.dcm', '1-02.dcm', '1-03.dcm', '1-04.dcm', '1-05.dcm', '1-06.dcm', '1-07.dcm', '1-08.dcm', '1-09.dcm', '1-10.dcm', '1-11.dcm', '1-12.dcm', '1-13.dcm', '1-14.dcm', '1-15.dcm', '1-16.dcm', '1-17.dcm', '1-18.dcm', '1-19.dcm', '1-20.dcm', '1-21.dcm', '1-22.dcm', '1-23.dcm', '1-24.dcm', '1-25.dcm', '1-26.dcm', '1-27.dcm', '1-28.dcm', '1-29.dcm', '1-30.dcm', '1-31.dcm', '1-32.dcm', '1-33.dcm', '1-34.dcm', '1-35.dcm', '1-36.dcm', '1-37.dcm', '1-38.dcm', '1-39.dcm', '1-40.dcm', '1-41.dcm', '1-42.dcm', '1-43.dcm', '1-44.dcm', '1-45.dcm', '1-46.dcm', '1-47.dcm', '1-48.dcm', '1-49.dcm', '1-50.dcm', '1-51.dcm', '1-52.dcm', '1-53.dcm', '1-54.dcm', '1-55.dcm', '1-56.dcm', '1-57.dcm', '1-58.dcm', '1-59.dcm', '1-60.dcm', '1-61.dcm', '1-62.dcm', '1-63.dcm', '1-64.dcm']


In [ ]:
import os
import pydicom
import cv2
import xml.etree.ElementTree as ET
from multiprocessing import Pool, cpu_count

# Define paths (NVMe SSD)
dicom_root = r"E:\FinalYearProject\Code\RaFoDATA\DatasetRF\dcm images\manifest-1608669183333\Lung-PET-CT-Dx"
annotation_root = r"E:\FinalYearProject\Code\RaFoDATA\DatasetRF\Annotation"
output_root = r"E:\FinalYearProject\Code\RaFoDATA\DatasetRF\Processed"

# Check if paths exist
if not os.path.exists(dicom_root):
    raise FileNotFoundError(f"The folder '{dicom_root}' does not exist.")
if not os.path.exists(annotation_root):
    raise FileNotFoundError(f"The folder '{annotation_root}' does not exist.")

# Create output folder if it doesn't exist
os.makedirs(output_root, exist_ok=True)

def load_dicom_image(dicom_path):
    """Load a DICOM image and return it as a NumPy array."""
    dicom = pydicom.dcmread(dicom_path)
    image = dicom.pixel_array
    return image

def parse_xml_annotation(xml_path):
    """Parse XML annotations and extract bounding boxes and labels."""
    tree = ET.parse(xml_path)
    root = tree.getroot()
    boxes = []
    labels = []
    for obj in root.findall('object'):
        label = obj.find('name').text
        bbox = obj.find('bndbox')
        xmin = int(bbox.find('xmin').text)
        ymin = int(bbox.find('ymin').text)
        xmax = int(bbox.find('xmax').text)
        ymax = int(bbox.find('ymax').text)
        boxes.append([xmin, ymin, xmax, ymax])
        labels.append(label)
    return boxes, labels

def convert_dicom_to_png(dicom_path, output_path):
    """Convert a DICOM image to PNG format and save it."""
    image = load_dicom_image(dicom_path)
    cv2.imwrite(output_path, image)

def find_xml_file(dicom_filename, annotation_root):
    """Recursively search for the corresponding XML file in subfolders."""
    for root, dirs, files in os.walk(annotation_root):
        for file in files:
            if file == dicom_filename.replace(".dcm", ".xml"):
                return os.path.join(root, file)
    return None

def process_file(args):
    dicom_path, annotation_root, output_root = args
    try:
        # Find the corresponding XML file
        xml_path = find_xml_file(os.path.basename(dicom_path), annotation_root)
        if not xml_path:
            print(f"Warning: No annotation found for {os.path.basename(dicom_path)}")
            return
        
        # Create output folder for the patient
        patient_folder = os.path.basename(os.path.dirname(os.path.dirname(dicom_path)))
        output_patient_folder = os.path.join(output_root, patient_folder)
        os.makedirs(output_patient_folder, exist_ok=True)
        
        # Convert DICOM to PNG
        output_image_path = os.path.join(output_patient_folder, os.path.basename(dicom_path).replace(".dcm", ".png"))
        convert_dicom_to_png(dicom_path, output_image_path)
        
        # Parse XML annotations
        boxes, labels = parse_xml_annotation(xml_path)
        print(f"Processed {os.path.basename(dicom_path)}: Boxes={boxes}, Labels={labels}")
    except Exception as e:
        print(f"Error processing {dicom_path}: {e}")

def find_dicom_files(dicom_root):
    """Recursively find all DICOM files in the given folder."""
    dicom_files = []
    for root, dirs, files in os.walk(dicom_root):
        for file in files:
            if file.endswith(".dcm"):
                dicom_files.append(os.path.join(root, file))
    return dicom_files

def process_dataset_parallel(dicom_root, annotation_root, output_root):
    """Process all DICOM images and XML annotations in parallel."""
    # Find all DICOM files
    dicom_files = find_dicom_files(dicom_root)
    print(f"Found {len(dicom_files)} DICOM files.")
    
    # Prepare tasks for parallel processing
    tasks = [(dicom_path, annotation_root, output_root) for dicom_path in dicom_files]
    
    # Use multiprocessing to process files in parallel
    with Pool(cpu_count()) as pool:
        pool.map(process_file, tasks)

# Run the parallel processing script
process_dataset_parallel(dicom_root, annotation_root, output_root)
print("Dataset processing complete!")

Found 251135 DICOM files.


In [ ]:
import os

dicom_root = r"E:\FinalYearProject\Code\RaFoDATA\DatasetRF\dcm images\manifest-1608669183333\Lung-PET-CT-Dx"
print(f"Checking folder: {dicom_root}")
if not os.path.exists(dicom_root):
    print(f"Error: The folder '{dicom_root}' does not exist.")
else:
    print(f"Folder exists. Contents:")
    for root, dirs, files in os.walk(dicom_root):
        print(f"Root: {root}")
        print(f"Subfolders: {dirs}")
        print(f"Files: {files}")

Checking folder: E:\LungCancerDataset\dcm images\manifest-1608669183333\Lung-PET-CT-Dx
Error: The folder 'E:\LungCancerDataset\dcm images\manifest-1608669183333\Lung-PET-CT-Dx' does not exist.
